In [24]:
import pandas as pd

In [25]:
df = pd.read_csv("dataset_3.csv")

In [26]:
df.columns

Index(['Authors', 'Author Full Names', 'Group Authors', 'Article Title',
       'Journals', 'Language', 'Document Type', 'Conference Title',
       'Conference Date', 'Conference Location', 'Conference Sponsor',
       'Author Keywords', 'Keywords Plus', 'Abstract', 'Addresses',
       'Country-Name', 'Leader Addresses', 'Email Addresses', 'Funding Orgs',
       'Funding Text', 'Cited Reference Count', 'Times Cited', 'Usage Count',
       'Publisher', 'Publisher City', 'ISSN', 'Publication Date',
       'Publication Year', 'DOI', 'Book DOI', 'Early Access Date',
       'Number of Pages', 'WoS Categories', 'Research Areas', 'IDS Number',
       'UT', 'Pubmed Id', 'Open Access Designations', 'Date of Export'],
      dtype='object')

# Leader Addresses Column

#### Data collected about leaders: 
- leader name
- leader country

In [27]:
leaderAddresses = df['Leader Addresses']

In [28]:
leaderAddresses[0]

'Poirier, RA (corresponding author), Mem Univ, Chem Dept, St John, NF A1B 3X7, Canada.'

### *getLeader( )*

In [29]:
def getLeader(text):
    data = {}
    data["leader_last_name"] = text.split(",",1)[0]
    data["leader_country"] = text.replace(".","").strip().split(" ")[-1]
    return data

In [30]:
getLeader(leaderAddresses[0])

{'leader_last_name': 'Poirier', 'leader_country': 'Canada'}

# Addresses Column

#### Data collected about authors : 
- firstname
- lastname
- fullname
- department
- university
- country

In [31]:
Addresses = df['Addresses']

In [32]:
Addresses[0]

'[Abu-Saleh, Abd Al-Aziz A.; Awad, Ibrahim E.; Poirier, Raymond A.] Mem Univ, Chem Dept, St John, NF A1B 3X7, Canada; [Yadav, Arpita] Chhatrapati Shahu Ji Maharaj Univ, Dept Chem, Univ Inst Engn & Technol, Kanpur 208024, Uttar Pradesh, India'

### *getAuthors( )*

In [33]:
def getAuthors(text):
    data = []
    addresses = text.split("[")[1:]
    for i in addresses:
        address = i.replace("\n","").strip();
        if address[-1] == ";":
            address = address[:-1]   
        address = address.split("]")    
        address_parts = address[1].split(",")
        university = address_parts[0]
        department = address_parts[1]
        country = address_parts[-1].split(" ")[-1]
        authors = address[0].replace(".","").split(";")
        for j in authors:
            author = {}
            name = j.replace("-"," ").split(",",1)
            author["first_name"] = name[-1].strip()
            author["last_name"] = name[0].strip()
            author["full_name"] = author["first_name"] + ", " + author["last_name"]
            author["department"] = department.replace("dept","").replace("Dept","").strip()
            author["university"] = university.strip()
            author["country"] = country.strip()
            data.append(author)
    return data

In [34]:
getAuthors(Addresses[0])

[{'first_name': 'Abd Al Aziz A',
  'last_name': 'Abu Saleh',
  'full_name': 'Abd Al Aziz A, Abu Saleh',
  'department': 'Chem',
  'university': 'Mem Univ',
  'country': 'Canada'},
 {'first_name': 'Ibrahim E',
  'last_name': 'Awad',
  'full_name': 'Ibrahim E, Awad',
  'department': 'Chem',
  'university': 'Mem Univ',
  'country': 'Canada'},
 {'first_name': 'Raymond A',
  'last_name': 'Poirier',
  'full_name': 'Raymond A, Poirier',
  'department': 'Chem',
  'university': 'Mem Univ',
  'country': 'Canada'},
 {'first_name': 'Arpita',
  'last_name': 'Yadav',
  'full_name': 'Arpita, Yadav',
  'department': 'Chem',
  'university': 'Chhatrapati Shahu Ji Maharaj Univ',
  'country': 'India'}]

# Collecting Authors and Leaders

In [35]:
authors = []
for i in Addresses:
    authors.append(getAuthors(i))
len(authors)

4145

In [36]:
leaders = []
count = 0
for i in leaderAddresses:
    leaders.append(getLeader(i))
len(leaders)

4145

# Adding Leaders to Authors

### *addLeaders( )*

In [37]:
def addLeaders(authors_data, leaders_data):
    result = []
    for authors,leader in zip(authors_data,leaders_data):
        leader_last_name = leader['leader_last_name']
        leader_name = ""
        temp = []
        for i in authors:
            if i['last_name'] == leader_last_name:
                leader_name = i['full_name']
        for j in authors:
            j['leader'] = leader_name
            temp.append(j)
        result.append(temp)
    return result

In [38]:
authors_row = addLeaders(authors,leaders)

In [39]:
authors_row[0]

[{'first_name': 'Abd Al Aziz A',
  'last_name': 'Abu Saleh',
  'full_name': 'Abd Al Aziz A, Abu Saleh',
  'department': 'Chem',
  'university': 'Mem Univ',
  'country': 'Canada',
  'leader': 'Raymond A, Poirier'},
 {'first_name': 'Ibrahim E',
  'last_name': 'Awad',
  'full_name': 'Ibrahim E, Awad',
  'department': 'Chem',
  'university': 'Mem Univ',
  'country': 'Canada',
  'leader': 'Raymond A, Poirier'},
 {'first_name': 'Raymond A',
  'last_name': 'Poirier',
  'full_name': 'Raymond A, Poirier',
  'department': 'Chem',
  'university': 'Mem Univ',
  'country': 'Canada',
  'leader': 'Raymond A, Poirier'},
 {'first_name': 'Arpita',
  'last_name': 'Yadav',
  'full_name': 'Arpita, Yadav',
  'department': 'Chem',
  'university': 'Chhatrapati Shahu Ji Maharaj Univ',
  'country': 'India',
  'leader': 'Raymond A, Poirier'}]

In [40]:
data = []
for i in authors_row:
    for j in i:
        data.append(j)        

# Adding API Names to Authors

In [41]:
# for i in data:
#     first_name = i["first_name"].split(" ")
#     if(len(first_name[0]) > 1):
#         i['api_name'] = first_name[0]
#     else:
#         last_name = i["last_name"].split(" ")
#         i['api_name'] = last_name[0]   

In [42]:
# file = open("data.txt","w")
# file.write(str(data))